In [ ]:
# !pip install undetected-chromedriver

In [1]:
import numpy as np
import pandas  as pd
import time
import traceback

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
def create_session_like_driver():
    options = uc.ChromeOptions()

    # user-agent + language
    options.add_argument(
        "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
    )
    options.add_argument("--lang=vi,en-US;q=0.9,en;q=0.8")

    # Một số option chống phát hiện cơ bản
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    # options.add_argument('--headless')   # bật nếu muốn chạy không mở cửa sổ

    # # Tắt load ảnh (cách đúng: dùng prefs)
    # prefs = {"profile.managed_default_content_settings.images": 2}
    # options.add_experimental_option("prefs", prefs)


    driver = uc.Chrome(options=options)
    return driver

# 1.Điền thông tin tk mk fb clon

In [3]:
email = "vnam95101@gmail.com"
password = "Njnjakhai01"

In [6]:
def login_fb(driver):
    driver.get("https://www.facebook.com/")
    wait = WebDriverWait(driver, 20)

    # dien thong tin tai khoan
    email_box = wait.until(EC.presence_of_element_located((By.ID, "email")))
    email_box.send_keys(email)
    time.sleep(1)

    # dien thong tin mk
    pass_box = wait.until(EC.presence_of_element_located((By.ID, "pass")))
    pass_box.send_keys(password)
    pass_box.send_keys(Keys.ENTER)

    wait.until(lambda d: "login" not in d.current_url.lower())

    time.sleep(20)

# Nhập link bài báo

In [7]:
list_link = [

    "https://www.facebook.com/NganhangPTIT/posts/pfbid0KrcrokJrCrUVih2px792P4TZNizq6jDUfpNXisDBeisyKFy8yMHPp2qfbiYRAZc6l",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid0ExzxEBd3PcyRzqPwcncwXx3hAktmZbFfow9zspRSFPvSFuohCFNzPxuKEtGfv16xl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid024mBxmwTGxpau5fcw6hNfyNhRttjyqXt4AakJygTSZDdwZGKgREM8TcWfsc2Uncmrl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid0X3mEmAp8bbPZsWY2tw9oYeRymhTTv1otwHynvtG2Ggu7C3dTZMgfiRXBibiKVvR3l",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid02HdXmRjfybsNzJeWmLTb6StePxKzKDFgYtbnPcj9RRxRXPmkpvFRkNLRjYwXNCqJkl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid02d6Yarv855zBGD54j7D7B8ELxw5BTN7VCp8HezWcSef7hkAhXFuSVT7fuTS55Dj2gl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid02DZXbBmh1PzPPJ7a7TBfUetJbxkjqcGpitXcE3bTefGem3NNz8kJNGr1mehsLn4gPl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid0CNWSiubc6fSeZJCYk5tvLE2tETr4pPtV78fbvQiPG64NfVGV1Y6bfkxYnHDvBkn8l",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid0zWNgCg7X1aNtUkiG7hTUmjthiGitgkZWEuKNAMmhrS8xoYiZkGTMrmaNybUC7eQCl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid02Bu6WApG1eSfgJJb7cQSeUgyBkhv8UPfB2PsPFFiXD1GKeEdJwEUCKk9uW2zZYMBkl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid0231d9LDumjvuV2pomYHX1FVR4msBAM8ckgmunDHkdPLAYFC4mGCxP1EZP8UzwaGYcl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid02kLtsGz6cP7y4DxcoouTqfnkWm2WKVcPfUCn9maZ1KztKPrVF8KE2yv5Ku8RDo3oUl",
    "https://www.facebook.com/NganhangPTIT/posts/pfbid0xuYcpQb5tnvRKkcbfqhsJy5hC652pbB93Yg7aSk3VPzcnw37Twc2ySuXYJVLjcvyl",
    "abc"
]


## Sau khi truy cập fb phải tắt thủ công thông báo và thông báo lưu mật khẩu vì sẽ làm hỏng tiến trình, có 20s sleep để tự tay tắt

In [10]:
# Basic setup
driver = create_session_like_driver()

login_fb(driver=driver)

for i, link in enumerate(list_link):
    print(f"Xử lý link {i + 1}")
    driver.get(link)
    wait = WebDriverWait(driver, 20)

    time.sleep(2)

    comment_button = wait.until(EC.element_to_be_clickable((
        By.XPATH,
        "/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[2]/div/div/div/div/div/div/div/div[2]/div[2]/div/div/div/div/div/div/div/div/div/div/div/div/div[13]/div/div/div[4]/div/div/div[2]/div[2]/div"
    )))
    comment_button.click()
    time.sleep(5)

    show_all_comment = wait.until(EC.presence_of_all_elements_located((
        By.XPATH, 
        "/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[3]/div/div/div[1]/div[1]/div/div/div/div/div/div/div[1]/div/div[3]"
    )))
    show_all_comment[0].click()
    time.sleep(5)

    # scroll_and_scraper_comment
    batch_size = 10
    previous_comment_count = 0
    no_change_count = 0
    comments = []

    for i in range(10):
        for batch in range(batch_size):
            # Scroll
            driver.execute_script("""
                document.querySelectorAll('*').forEach(el => {
                    if (el.scrollHeight > el.clientHeight) {
                        el.scrollTop += 1200;
                    }
                });
            """)
            time.sleep(3)
            
            # Đếm comments hiện tại
            current_comments = driver.find_elements(By.CSS_SELECTOR, 'span[dir="auto"]')
            current_count = len(current_comments)
            
            print(f"Batch {i+1} - lần {batch+1}: {current_count} comments")
            
            # Kiểm tra nếu đếm comment trong batch không tăng, chuyển sang batch tiếp
            if current_count == previous_comment_count:
                no_change_count += 1
                if no_change_count >= 3:  # 3 lần liên tiếp không tăng
                    print("✅ Không có comment mới trong batch")
                    break
            else:
                no_change_count = 0  # Reset counter
            
            previous_comment_count = current_count
            
            # Scraper comment và lưu ngay trong batch
            current_container_comments = driver.find_elements(
                By.CSS_SELECTOR,
                'span[dir="auto"]'
            )

            for container in current_container_comments:
                div_elements = container.find_elements(
                    By.CSS_SELECTOR,
                    'div[dir="auto"]'
                )
                
                full_text = []
                for div in div_elements:
                    text = div.text.strip()
                    if text:
                        full_text.append(text)  

                if full_text:
                    combine_text = '\n'.join(full_text)
                    if len(combine_text) > 5:
                        comments.append({
                            'text': combine_text,
                            'length': len(combine_text)
                        })

    time.sleep(20)

    print("-----------------------------------------------------")

    # Lọc ra comment unique trong list comment trùng nhau
    unique_comments = []
    seen_comments = set()

    for item in comments:
        if item['text'] not in seen_comments:
            unique_comments.append(item)
            seen_comments.add(item['text'])

    print(f"Tổng số comment unique: {len(unique_comments)}")

    df = pd.DataFrame(unique_comments)
    df.to_csv("comment_data.csv", mode='a', header=False, index=False)

    print("✅ Đã lưu comment vào CSV")

driver.quit()

Xử lý link 1
Batch 1 - lần 1: 181 comments
Batch 1 - lần 2: 191 comments
Batch 1 - lần 3: 191 comments
Batch 1 - lần 4: 191 comments
Batch 1 - lần 5: 191 comments
✅ Không có comment mới trong batch
Batch 2 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 3 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 4 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 5 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 6 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 7 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 8 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 9 - lần 1: 191 comments
✅ Không có comment mới trong batch
Batch 10 - lần 1: 191 comments
✅ Không có comment mới trong batch
-----------------------------------------------------
Tổng số comment unique: 26
✅ Đã lưu comment vào CSV
Xử lý link 2
Batch 1 - lần 1: 156 comments
Batch 1 - lần 2: 166 comments
Batch 1 - lần 3: 166 comments
Batch 1

InvalidArgumentException: Message: invalid argument
  (Session info: chrome=141.0.7390.66)
Stacktrace:
	GetHandleVerifier [0x0x116fea3+66515]
	GetHandleVerifier [0x0x116fee4+66580]
	(No symbol) [0x0xf5da6b]
	(No symbol) [0x0xf4da70]
	(No symbol) [0x0xf4c061]
	(No symbol) [0x0xf4c7fb]
	(No symbol) [0x0xf617ce]
	(No symbol) [0x0xfed7f7]
	(No symbol) [0x0xfcb1cc]
	(No symbol) [0x0xfecba7]
	(No symbol) [0x0xfcafc6]
	(No symbol) [0x0xf9c2ca]
	(No symbol) [0x0xf9d154]
	GetHandleVerifier [0x0x13c73d3+2521347]
	GetHandleVerifier [0x0x13c2353+2500739]
	GetHandleVerifier [0x0x1197cf4+229924]
	GetHandleVerifier [0x0x1188258+165768]
	GetHandleVerifier [0x0x118ed0d+193085]
	GetHandleVerifier [0x0x11781b8+100072]
	GetHandleVerifier [0x0x1178350+100480]
	GetHandleVerifier [0x0x116260a+11066]
	BaseThreadInitThunk [0x0x75dd5d49+25]
	RtlInitializeExceptionChain [0x0x772ed6db+107]
	RtlGetAppContainerNamedObjectPath [0x0x772ed661+561]


In [11]:
test = pd.read_csv("comment_data.csv")
test

,text,length
0,CHÚC MỪNG SINH VIÊN PTIT TỎA SÁNG VỚI BA GIẢI ...,2865
1,Chúc mừng zai mẹ nhé,20
2,Xin chúc mừng các bạn trẻ,25
3,1 bữa lẩu ở iec nhé\nNguyễn Trang,32
4,Nguyễn Trang\nchờ quà hcm lâu quá,32
...,...,...
4744,Đâu:)?,6
4745,Hoang Anh\n?,11
4746,Chi Linh,8
4747,Quynh Anh\nok,12
